In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
# Load the raw IoT dataset
data_path = "../data/raw/RT_IOT2022"
df = pd.read_csv(data_path)

# Check the shape of the raw data
df.shape

(123117, 85)

In [3]:
def drop_unused_columns(df):
    """
    Removes unnecessary columns such as index columns created during export.
    """
    
    # Create a copy so we don't modify the original dataframe
    df = df.copy()
    
    # Drop the unnamed index column if it exists
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    
    return df

In [4]:
def fix_dtypes(df):
    """
    Converts categorical columns to category type for better memory usage and modeling.
    """
    
    df = df.copy()
    
    # Columns that should be treated as categorical
    cat_cols = ["proto", "service"]
    
    # Convert each categorical column if it exists
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype("category")
    
    return df

In [5]:
def clean_missing_and_duplicates(df):
    """
    Removes duplicate rows and fills missing numeric values.
    """
    
    df = df.copy()
    
    # Remove duplicate rows
    df = df.drop_duplicates()
    
    # Select only numeric columns
    num_cols = df.select_dtypes(include="number").columns
    
    # Forward fill missing numeric values
    df[num_cols] = df[num_cols].fillna(method="ffill")
    
    return df

In [6]:
def select_features(df):
    """
    Selects a subset of important features for modeling and dashboard use.
    """
    
    # Core features related to traffic behavior
    features = [
        "flow_duration",
        "fwd_pkts_tot",
        "bwd_pkts_tot",
        "fwd_pkts_per_sec",
        "bwd_pkts_per_sec",
        "proto",
        "service"
    ]
    
    # Automatically add any label or attack column if present
    label_cols = [col for col in df.columns if "label" in col.lower() or "attack" in col.lower()]
    
    # Combine features with label columns
    features = features + label_cols
    
    return df[features]

In [8]:
def clean_missing_and_duplicates(df):
    """
    Removes duplicate rows and fills missing numeric values.
    """
    
    df = df.copy()
    
    # Remove duplicate rows
    df = df.drop_duplicates()
    
    # Select only numeric columns
    num_cols = df.select_dtypes(include="number").columns
    
    # Forward fill missing numeric values
    df[num_cols] = df[num_cols].ffill()
    
    return df

In [9]:
# Display information about the cleaned dataset
df_clean.info()

# Check for any remaining missing values
df_clean.isna().sum()

# Show basic statistics for numeric columns
df_clean.describe()

<class 'pandas.DataFrame'>
RangeIndex: 123117 entries, 0 to 123116
Data columns (total 84 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   id.orig_p                 123117 non-null  int64   
 1   id.resp_p                 123117 non-null  int64   
 2   proto                     123117 non-null  category
 3   service                   123117 non-null  category
 4   flow_duration             123117 non-null  float64 
 5   fwd_pkts_tot              123117 non-null  int64   
 6   bwd_pkts_tot              123117 non-null  int64   
 7   fwd_data_pkts_tot         123117 non-null  int64   
 8   bwd_data_pkts_tot         123117 non-null  int64   
 9   fwd_pkts_per_sec          123117 non-null  float64 
 10  bwd_pkts_per_sec          123117 non-null  float64 
 11  flow_pkts_per_sec         123117 non-null  float64 
 12  down_up_ratio             123117 non-null  float64 
 13  fwd_header_size_tot       123117 non-nul

,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size
count,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,1.231170e+05,1.231170e+05,1.231170e+05,...,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,123117.000000,123117.000000,123117.000000
mean,34639.258738,1014.305092,3.809566,2.268826,1.909509,1.471218,0.820260,3.518063e+05,3.517620e+05,7.035683e+05,...,1.481354e+05,2.353599e+04,1.616655e+06,1.701956e+06,3.517644e+06,1.664985e+06,4.550183e+04,6118.905123,2739.776018,751.647514
std,19070.620354,5256.371994,130.005408,22.336565,33.018311,19.635196,32.293948,3.707645e+05,3.708015e+05,7.415634e+05,...,1.613007e+06,1.477935e+06,8.809396e+06,9.252337e+06,1.229508e+08,9.007064e+06,1.091361e+06,18716.313861,10018.848534,6310.183843
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,17702.000000,21.000000,0.000001,1.000000,1.000000,1.000000,0.000000,7.454354e+01,7.288927e+01,1.490871e+02,...,9.536740e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
50%,37221.000000,21.000000,0.000004,1.000000,1.000000,1.000000,0.000000,2.467238e+05,2.467238e+05,4.934475e+05,...,4.053116e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
75%,50971.000000,21.000000,0.000005,1.000000,1.000000,1.000000,0.000000,5.242880e+05,5.242880e+05,1.048576e+06,...,5.006790e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
max,65535.000000,65389.000000,21728.335578,4345.000000,10112.000000,4345.000000,10105.000000,1.048576e+06,1.048576e+06,2.097152e+06,...,4.374931e+08,4.774862e+08,3.000000e+08,3.000000e+08,2.096777e+10,3.000000e+08,1.208029e+08,65535.000000,65535.000000,65535.000000


In [10]:
# Save the cleaned dataset for modeling and dashboard use
output_path = "../data/processed/clean_iot_data.csv"
df_clean.to_csv(output_path, index=False)

output_path

'../data/processed/clean_iot_data.csv'